In [1]:
import pandas as pd
import numpy as np
import requests, re
import datetime
import time
from KrakenInterface import KrakenInterface, format_codes_names
from bs4 import BeautifulSoup

In [3]:
page = open('curr_names.html', 'r').read()
soup = BeautifulSoup(page, "lxml")
table = soup.findAll("td")[4:]
for status in table[3::4][:6]:
    print(status.contents[0])
    print(status.contents[0] == '\xa0')

 
True
 
True
 
True
 
True
 
True
<span class='3D"wysiwyg-color-red120"'>delisted</span>
False


In [4]:
codenames = {}
for code, name, status in zip(table[::4], table[1::4], table[3::4]):
    if status.contents[0] == '\xa0':
        if code.span is None:
                codenames[code.contents[0]] = name.contents[0]
        else:
            codenames[code.span.contents[0]] = re.findall("[\w\s]+", name.span.contents[0])[0]
    
#     

codenames

{'ADA': 'Cardano',
 'ALGO': 'Algorand',
 'ATOM': 'Cosmos',
 'BAT': 'Basic Attention Token',
 'BCH': 'Bitcoin Cash',
 'COMP': 'Compound',
 'DAI': 'Dai',
 'DASH': 'DASH',
 'DOT': 'DOT',
 'EOS': 'EOS',
 'GNO': 'Gnosis',
 'ICX': 'ICON',
 'KAVA': 'Kava',
 'KNC': 'Kyber Network',
 'LINK': 'Chainlink',
 'LSK': 'Lisk',
 'OMG': 'OmiseGO',
 'OXT': 'Orchid',
 'PAXG': 'PAX Gold',
 'QTUM': 'QTUM',
 'SC': 'Siacoin',
 'STORJ': 'Storj',
 'TRX': 'Tron',
 'USDT': 'Tether',
 'USDC': 'USD Coin',
 'WAVE': 'Waves',
 'XETC': 'Ethereum Classic',
 'XETH': 'Ethereum',
 'XLTC': 'Litecoin',
 'XMLN': 'Watermelon',
 'XREP': 'Augur',
 'XREPV2': 'Augur v2',
 'XXBT': 'Bitcoin',
 'XXDG': 'Dogecoin',
 'XXLM': 'Stellar Lumens',
 'XXMR': 'Monero',
 'XXRP': 'Ripple',
 'XXTZ': 'Tezos',
 'XZEC': 'Zcash',
 'ZCAD': 'Canadian dollar',
 'ZEUR': 'Euro',
 'ZGBP': 'Great British Pound',
 'ZJPY': 'Japanese Yen',
 'ZUSD': 'US Dollar',
 'KFEE': 'Kraken Fee Credits'}

In [5]:
# codes_clean = ass.code.apply(lambda x: x[1:] if (x[0] == "X")|(x[0] == "Z") else x)

ass = pd.DataFrame(data=codenames.items(), columns=['codes', 'name'])
ass["codes_clean"] = ass.codes.apply(lambda x: x[1:] if (x[0] == "X")|(x[0] == "Z") else x)
ass["fiats"] = ass.codes.apply(lambda x: x[1:] if x[0] == 'Z' else np.nan)
ass["cryptos"] = ass[ass.fiats.isna()]["codes_clean"]


fiat_codes = ass["fiats"].dropna().values
fiat_codes

array(['CAD', 'EUR', 'GBP', 'JPY', 'USD'], dtype=object)

In [6]:
ass["fiats"].dropna().values

array(['CAD', 'EUR', 'GBP', 'JPY', 'USD'], dtype=object)

In [7]:
ass.codes.values

array(['ADA', 'ALGO', 'ATOM', 'BAT', 'BCH', 'COMP', 'DAI', 'DASH', 'DOT',
       'EOS', 'GNO', 'ICX', 'KAVA', 'KNC', 'LINK', 'LSK', 'OMG', 'OXT',
       'PAXG', 'QTUM', 'SC', 'STORJ', 'TRX', 'USDT', 'USDC', 'WAVE',
       'XETC', 'XETH', 'XLTC', 'XMLN', 'XREP', 'XREPV2', 'XXBT', 'XXDG',
       'XXLM', 'XXMR', 'XXRP', 'XXTZ', 'XZEC', 'ZCAD', 'ZEUR', 'ZGBP',
       'ZJPY', 'ZUSD', 'KFEE'], dtype=object)

In [8]:
asset = 'XBT'
fiat = 'EUR'
kr = KrakenInterface()
asks, bids = kr.order_book_of(10)
# rec = requests.get('https://api.kraken.com/0/public/Ticker?pair=XBTeur').json()['result']
asks

,price,volume,timestamp
0,9240.000000,0.394,2020-09-27 21:54:40
1,9242.500000,0.014,2020-09-27 21:54:40
2,9243.700195,0.198,2020-09-27 21:54:40
3,9243.799805,0.097,2020-09-27 21:54:40
4,9244.400391,0.932,2020-09-27 21:54:40
5,9244.700195,0.373,2020-09-27 21:54:40
6,9244.799805,1.936,2020-09-27 21:54:40
7,9244.900391,0.932,2020-09-27 21:54:40
8,9245.599609,0.014,2020-09-27 21:54:40
9,9245.799805,0.107,2020-09-27 21:54:40


In [9]:
kr.asset = 'ETH'
kr.order_book_of(1)[0].price.iloc[-1]

304.72

In [10]:
# s = ''
# for key, val in zip(asset_names.codes.values, asset_names.name.values):
#     s += '{:<4s} | {:s}\n'.format(str(key), str(val))
    
# print(s)

In [11]:
kr.get_asset_pairs()["quote"].unique()

array(['XETH', 'ZEUR', 'ZUSD', 'XXBT', 'ZJPY', 'ZAUD', 'ZGBP', 'USDT',
       'CHF', 'DAI', 'USDC', 'ZCAD'], dtype=object)

In [12]:
kr.get_fiat_pairs()["quote"].unique()

array(['ZEUR', 'ZUSD', 'ZJPY', 'ZGBP', 'ZCAD'], dtype=object)

In [13]:
unknown_ass = "LINK"
kr.get_fiat_pairs().query("@unknown_ass in base")

,altname,wsname,aclass_base,base,aclass_quote,quote,base_clean,quote_clean
LINKEUR,LINKEUR,LINK/EUR,currency,LINK,currency,ZEUR,LINK,EUR
LINKUSD,LINKUSD,LINK/USD,currency,LINK,currency,ZUSD,LINK,USD


In [14]:
asset, currency = "XRP", "EUR"
# asset, currency = "LINK", "USD"
base = 'https://api.kraken.com/0/public/'
market = asset + currency
url = base + 'Depth?pair={}&count={}'.format(market.upper(), 5)

page = requests.get(url)
jString = page.json()
jString

{'error': [],
 'result': {'XXRPZEUR': {'asks': [['0.20787000', '1002.050', 1601236545],
    ['0.20788000', '10000.000', 1601236507],
    ['0.20791000', '3075.000', 1601236504],
    ['0.20794000', '1522.810', 1601236545],
    ['0.20795000', '1700.000', 1601236544]],
   'bids': [['0.20782000', '22392.731', 1601236546],
    ['0.20781000', '12000.000', 1601236543],
    ['0.20780000', '1499.990', 1601236544],
    ['0.20779000', '10000.000', 1601236545],
    ['0.20777000', '3121.000', 1601236494]]}}}

In [15]:
asset_pairs = kr.get_asset_pairs()
asset_pairs.query("base_clean == @asset & quote_clean == @currency")

,altname,wsname,aclass_base,base,aclass_quote,quote,base_clean,quote_clean
XXRPZEUR,XRPEUR,XRP/EUR,currency,XXRP,currency,ZEUR,XRP,EUR


In [16]:
asset_pairs.query("base_clean == @asset")["quote_clean"].values

array(['AUD', 'ETH', 'GBP', 'USDT', 'XBT', 'CAD', 'EUR', 'JPY', 'USD'],
      dtype=object)

In [17]:
asset_pairs = kr.get_asset_pairs()
valid = asset_pairs.query("base_clean == @asset").quote_clean.values
valid

array(['AUD', 'ETH', 'GBP', 'USDT', 'XBT', 'CAD', 'EUR', 'JPY', 'USD'],
      dtype=object)

In [18]:
# print(format_codes_names(kr.assets))

In [19]:
code= "XBT"
kr.assets.query("codes_clean == @code").name.values[0]

'Bitcoin'

In [20]:
kr.assets.query("codes_clean == 'ADA'").name[0]

'Cardano'

In [21]:
fiat_pairs = kr.get_fiat_pairs()
valid = fiat_pairs.query("base_clean == @asset")
# valid.quote_clean
valid

,altname,wsname,aclass_base,base,aclass_quote,quote,base_clean,quote_clean
XRPGBP,XRPGBP,XRP/GBP,currency,XXRP,currency,ZGBP,XRP,GBP
XXRPZCAD,XRPCAD,XRP/CAD,currency,XXRP,currency,ZCAD,XRP,CAD
XXRPZEUR,XRPEUR,XRP/EUR,currency,XXRP,currency,ZEUR,XRP,EUR
XXRPZJPY,XRPJPY,XRP/JPY,currency,XXRP,currency,ZJPY,XRP,JPY
XXRPZUSD,XRPUSD,XRP/USD,currency,XXRP,currency,ZUSD,XRP,USD


In [22]:
def print_codes_names(df, column):
    s = ''
    max_len = df[column].str.len().max()
    for key, val in zip(df[column].values, df.name.values):
        s += f'{str(key):<{max_len}s} | {str(val):s}\n'
    return s

print(print_codes_names(kr.assets, "codes"))

ADA    | Cardano
ALGO   | Algorand
ATOM   | Cosmos
BAT    | Basic Attention Token
BCH    | Bitcoin Cash
COMP   | Compound
DAI    | Dai
DASH   | DASH
DOT    | DOT
EOS    | EOS
GNO    | Gnosis
ICX    | ICON
KAVA   | Kava
KNC    | Kyber Network
LINK   | Chainlink
LSK    | Lisk
OMG    | OmiseGO
OXT    | Orchid
PAXG   | PAX Gold
QTUM   | QTUM
SC     | Siacoin
STORJ  | Storj
TRX    | Tron
USDT   | Tether
USDC   | USD Coin
WAVE   | Waves
XETC   | Ethereum Classic
XETH   | Ethereum
XLTC   | Litecoin
XMLN   | Watermelon
XREP   | Augur
XREPV2 | Augur v2
XXBT   | Bitcoin
XXDG   | Dogecoin
XXLM   | Stellar Lumens
XXMR   | Monero
XXRP   | Ripple
XXTZ   | Tezos
XZEC   | Zcash
ZCAD   | Canadian dollar
ZEUR   | Euro
ZGBP   | Great British Pound
ZJPY   | Japanese Yen
ZUSD   | US Dollar
KFEE   | Kraken Fee Credits



In [23]:
fiat_pairs = kr.get_fiat_pairs()
valid = fiat_pairs.query("base_clean == @asset")
# valid.quote_clean
valid

,altname,wsname,aclass_base,base,aclass_quote,quote,base_clean,quote_clean
XRPGBP,XRPGBP,XRP/GBP,currency,XXRP,currency,ZGBP,XRP,GBP
XXRPZCAD,XRPCAD,XRP/CAD,currency,XXRP,currency,ZCAD,XRP,CAD
XXRPZEUR,XRPEUR,XRP/EUR,currency,XXRP,currency,ZEUR,XRP,EUR
XXRPZJPY,XRPJPY,XRP/JPY,currency,XXRP,currency,ZJPY,XRP,JPY
XXRPZUSD,XRPUSD,XRP/USD,currency,XXRP,currency,ZUSD,XRP,USD


In [24]:
# fiat_pairs[["base_clean", "quote_clean"]] = fiat_pairs.wsname.str.split("/", expand=True)
# fiat_pairs

In [25]:
marketS = find_index(asset.upper(), currency.upper())
marketS = '{}{}'.format(asset.upper(), currency.upper())
asks = jString["result"][marketS]["asks"]
bids = jString["result"][marketS]["bids"]

dataAsks = pd.DataFrame(asks, columns=['price', 'volume', 'timestamp'], dtype='float32')
dataBids = pd.DataFrame(bids, columns=['price', 'volume', 'timestamp'], dtype='float32')

dataAsks

NameError: name 'find_index' is not defined

In [26]:
kr.fiats[["codes_clean", "name"]].values

array([['CAD', 'Canadian dollar'],
       ['EUR', 'Euro'],
       ['GBP', 'Great British Pound'],
       ['JPY', 'Japanese Yen'],
       ['USD', 'US Dollar']], dtype=object)

In [27]:
kr.fiats.codes_clean.values

array(['CAD', 'EUR', 'GBP', 'JPY', 'USD'], dtype=object)

In [28]:
kr.assets

,codes,name,codes_clean,fiats,cryptos
0,ADA,Cardano,ADA,NaN,ADA
1,ALGO,Algorand,ALGO,NaN,ALGO
2,ATOM,Cosmos,ATOM,NaN,ATOM
3,BAT,Basic Attention Token,BAT,NaN,BAT
4,BCH,Bitcoin Cash,BCH,NaN,BCH
5,COMP,Compound,COMP,NaN,COMP
6,DAI,Dai,DAI,NaN,DAI
7,DASH,DASH,DASH,NaN,DASH
8,DOT,DOT,DOT,NaN,DOT
9,EOS,EOS,EOS,NaN,EOS


In [29]:
fiat_pairs.base.isin(kr.fiats.codes)

ADAEUR      False
ADAUSD      False
ALGOEUR     False
ALGOUSD     False
ATOMEUR     False
            ...  
XZECZUSD    False
ZEURZUSD     True
ZGBPZUSD     True
ZUSDZCAD     True
ZUSDZJPY     True
Name: base, Length: 111, dtype: bool

In [30]:
kr.fiat_codes

array(['CAD', 'EUR', 'GBP', 'JPY', 'USD'], dtype=object)

In [33]:
allpairs = kr.get_asset_pairs()
allpairs.query("@ass.codes.values not in base")

,altname,wsname,aclass_base,base,aclass_quote,quote,base_clean,quote_clean
AUDJPY,AUDJPY,AUD/JPY,currency,ZAUD,currency,ZJPY,AUD,JPY
AUDUSD,AUDUSD,AUD/USD,currency,ZAUD,currency,ZUSD,AUD,USD
BALETH,BALETH,BAL/ETH,currency,BAL,currency,XETH,BAL,ETH
BALEUR,BALEUR,BAL/EUR,currency,BAL,currency,ZEUR,BAL,EUR
BALUSD,BALUSD,BAL/USD,currency,BAL,currency,ZUSD,BAL,USD
BALXBT,BALXBT,BAL/XBT,currency,BAL,currency,XXBT,BAL,XBT
CRVETH,CRVETH,CRV/ETH,currency,CRV,currency,XETH,CRV,ETH
CRVEUR,CRVEUR,CRV/EUR,currency,CRV,currency,ZEUR,CRV,EUR
CRVUSD,CRVUSD,CRV/USD,currency,CRV,currency,ZUSD,CRV,USD
CRVXBT,CRVXBT,CRV/XBT,currency,CRV,currency,XXBT,CRV,XBT


In [36]:
allpairs

,altname,wsname,aclass_base,base,aclass_quote,quote,base_clean,quote_clean
ADAETH,ADAETH,ADA/ETH,currency,ADA,currency,XETH,ADA,ETH
ADAEUR,ADAEUR,ADA/EUR,currency,ADA,currency,ZEUR,ADA,EUR
ADAUSD,ADAUSD,ADA/USD,currency,ADA,currency,ZUSD,ADA,USD
ADAXBT,ADAXBT,ADA/XBT,currency,ADA,currency,XXBT,ADA,XBT
ALGOETH,ALGOETH,ALGO/ETH,currency,ALGO,currency,XETH,ALGO,ETH
...,...,...,...,...,...,...,...,...
XZECZUSD,ZECUSD,ZEC/USD,currency,XZEC,currency,ZUSD,ZEC,USD
ZEURZUSD,EURUSD,EUR/USD,currency,ZEUR,currency,ZUSD,EUR,USD
ZGBPZUSD,GBPUSD,GBP/USD,currency,ZGBP,currency,ZUSD,GBP,USD
ZUSDZCAD,USDCAD,USD/CAD,currency,ZUSD,currency,ZCAD,USD,CAD


In [39]:
fiatnames = ass[ass.codes.str.startswith('Z')]
fiatnames

,codes,name,codes_clean,fiats,cryptos
39,ZCAD,Canadian dollar,CAD,CAD,NaN
40,ZEUR,Euro,EUR,EUR,NaN
41,ZGBP,Great British Pound,GBP,GBP,NaN
42,ZJPY,Japanese Yen,JPY,JPY,NaN
43,ZUSD,US Dollar,USD,USD,NaN


In [40]:
asset_names = re.findall('<tr>\n<td class="confluenceTd">(\w*)</td>\n<td class="confluenceTd">(.*)</td>', tbody)
asset_names = pd.DataFrame(asset_names, columns=['codes', 'name'])
fiatnames = asset_names[asset_names.codes.isin(fiats)]
codes_clean = asset_names.codes.apply(lambda x: x[1:] if (x[0] == "X")|(x[0] == "Z") else x)
# asset_names.codes = codes_clean
asset_names[asset_names.codes.str.startswith('Z')]
# .sort_values('codes')
fiatnames

NameError: name 'tbody' is not defined

In [37]:
pd.set_option("display.max_rows", 50, "display.max_columns", 10)
allpairs[allpairs.base.isin(codes_clean)]

NameError: name 'codes_clean' is not defined

In [38]:
# page = open(r'curr_names.html', 'r').read()
# soup = BeautifulSoup(page, "html")
# tbody = str(soup.find('tbody'))

In [ ]:
# url = 'https://api.kraken.com/0/public/AssetPairs'
# page = requests.get(url)
# jString = page.json()
# jString

In [ ]:
# allpairs = jString["result"].keys()
# allpairs
# # [i for i in allpairs if "EUR" in i]

In [ ]:
fiats = fiatnames.codes.values 
fiats

In [ ]:
fiatmask = allpairs['quote'].isin(fiats)
fiatpairs = allpairs[fiatmask]
fiatpairs

In [ ]:
cryptofiat = fiatpairs.base.unique()
cryptofiat

In [ ]:
cryptofiat

In [ ]:
asset_names[asset_names.codes.isin(cryptofiat)]

In [ ]:
allpairs.quote.unique()

In [ ]:
allpairs[allpairs.base == 'XXBT']